In [ ]:
#imports 
import json
import os
import requests
import pandas as pd
import matplotlib as plt
import pprint as pprint
from pandas.io.json import json_normalize
import gmaps
import gmaps.datasets

In [ ]:
# data file to be saved
rawData = "../data/rawData/zl_normalized_dataframe.csv"


# renamed data file to be saved
orgRawData = "../data/rawData/zl_organized_dataframe.csv"

# cleaned data file to be saved
cleanData = "../data/cleanData/zl_clean_data.csv"

# path csv file location
pathRawData = "../data/rawData/"
pathCleanData ="../datacleanData/"

In [ ]:
# Remember to update the config file with your API key
from config import api_key
from config import api_id
from config import gkey

In [ ]:
url = "https://api.adzuna.com/v1/api/jobs/us/search/"
#have to create iteration to get all pages of data
page = "1"
api_details ="?" + "app_id=" + api_id + "&app_key=" + api_key
# Build query URL
query_url = url + page + api_details
query_url

In [ ]:
data_response = requests.get(query_url)
data_json = data_response.json()
type(data_json)


In [ ]:
#Getting data using json_normalize
#LINK: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html

pages_test = [str(x) for x in range(1,101)]

counter = 0
# Build query URL
data_pages_df = pd.DataFrame()

for page_num in pages_test:
    response = requests.get(url + page_num + api_details).json()
    
    #create a df to store the normalized page that has the job postings
    normalized_page = json_normalize(response['results'])
    
    #add normalized data into a df
    data_page_df = pd.DataFrame(normalized_page)
    
    #add the url column to df incase an error occurs you know what page you were on
    data_page_df['query_url'] = 'page number ' + page_num
    
    #append to a new df so each page can be saved
    #Sorting because non-concatenation axis is not aligned
    data_pages_df = data_pages_df.append(data_page_df, sort=True)
    
    counter += 1
    print(f"Page {page_num} iteration complete")
    
print(f'total rows iterated | {counter * 10}')   
      

In [ ]:
#save raw data to dataRaw folder
data_pages_df.to_csv(rawData)

In [ ]:
#calling the csv raw data
csv_file = pd.read_csv(rawData)
csv_file.head()


In [ ]:
#making a copy
csv_file_copy = csv_file.copy()

#getting only the rows that we are using
csv_file_copy = csv_file_copy[["id","title","category.label","company.display_name", "location.display_name", "latitude","longitude" ]]
csv_file_copy.head()

In [ ]:
#Renaming the column names

renamed_csv_file = csv_file_copy.rename(columns = {
    "id": "Job Posting ID",
    "title": "Job Title",
    "category.label":"Category",
    "company.display_name":"Company Name",
    "location.display_name":"Location",
    "latitude":"Lat",
    "longitude":"Lng"
})
renamed_csv_file.head()

In [ ]:
#save raw data to dataRaw folder
renamed_csv_file.to_csv(orgRawData)

In [ ]:
#calling the csv org data
org_csv_file = pd.read_csv(orgRawData)
org_csv_file.head()

In [ ]:
#Count Rows of all columns
org_csv_file.count()

In [ ]:
#Remove unwanted column
del org_csv_file['Unnamed: 0']
org_csv_file.head()

In [ ]:
#Delete Rows with missing data from lat and lng
clean_data = org_csv_file.dropna()
clean_data.count()

In [ ]:
#Save clean data into cleanData folder
clean_data.to_csv(cleanData)

In [ ]:
#Deleting rows with any missing data
#calling the csv org data
clean_csv_file = pd.read_csv(cleanData)

#Not sure why this column came back
del clean_csv_file['Unnamed: 0']

clean_csv_file.head()

In [ ]:
#Calling Gmaps key
gmaps.configure(api_key= gkey) # Fill in with your API key
clean_csv_file_copy = clean_csv_file.copy()

#defining rows
lat = clean_csv_file_copy['Lat']
lng = clean_csv_file_copy['Lng']
cat = clean_csv_file_copy['Job Posting ID']

#Turning int to float for testing
cat = cat.astype('float')

In [ ]:
#Gmaps testing
locations = clean_csv_file_copy[['Lat', 'Lng']]
fig = gmaps.figure(map_type='TERRAIN')



locations_layer = gmaps.symbol_layer(
    locations, fill_color='red', stroke_color='green', scale=1
)
fig = gmaps.figure()
fig.add_layer(locations_layer)
fig

In [ ]:
#Geting unique values
charr = clean_csv_file_copy['Category'].unique()
charr

In [ ]:
#categorizing category to display on map layter
index_cat = clean_csv_file_copy.set_index(['Category'])
cat = index_cat.loc["Sales Jobs"] 
healthcare = cat[['Lat', 'Lng']]
healthcare

# for c in range(len(charr)):
#     flag = index_cat.loc[c]
#     arm = flag[['Lat','Lng']]
    
clean_csv_file_copy